In [1]:
import gymnasium as gym
from gymnasium.spaces import Discrete, Box, Sequence, Dict
import numpy as np
from src.commons import methods
import statistics

from src.rl_environments.BaseEnv import BaseEnv as Env

import os

## Config

In [2]:
TRAIN_CONFIG = {
    'alpha_range': [ (0 + i / 10) for i in range(1,10)],
    'beta_range': [round(i * 0.5, 1) for i in range(2, 6)],
    'h_range': [0.5],
    'c_range': [25],
    'total': range(10, 40),
    'travel_time': 'uniform',
}

TEST_CONFIG = {
    'alpha_range': [ (3 + i / 10) for i in range(1,10)],
    'beta_range': [round(i * 0.5, 1) for i in range(2, 6)],
    'h_range': [0.5],
    'c_range': [25],
    'total': range(10, 40), 
    'travel_time': 'uniform',
}



In [3]:
PPO_MODEL_NAME = 'PPO_Env1'

## Train PQN

In [4]:
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv

if True:

    n_cpus = os.cpu_count()

    print(f"Number of processors: {n_cpus}")
    env = make_vec_env(lambda: Env(config=TRAIN_CONFIG), n_envs=n_cpus, vec_env_cls=SubprocVecEnv)
    model = PPO("MlpPolicy", env, verbose=1, device='cpu')

    print(model.n_envs)

    total_timesteps = 1_000
    print(f"Training model for {total_timesteps} timesteps")
    # Start training
    model.learn(total_timesteps=total_timesteps)

    model.save(methods.file_path(PPO_MODEL_NAME, 'models'))



Number of processors: 8
Using cpu device
8
Training model for 1000 timesteps
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 4.18     |
|    ep_rew_mean     | -5.7     |
| time/              |          |
|    fps             | 9532     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 16384    |
---------------------------------


## Generate Test Data

In [5]:
def get_realized_data(config):
    alpha = np.random.choice(config['alpha_range'])
    beta = np.random.choice(config['beta_range'])
    h = np.random.choice(config['h_range'])
    c = np.random.choice(config['c_range'])
    total = np.random.choice(config['total'])
    intervals = np.random.gamma(shape=alpha, scale=beta, size=total)
    # travel_time = sum(intervals[4:]) - np.random.exponential(scale=alpha * beta)
    travel_time = sum(intervals[4:]) - np.random.gamma(shape=4, scale=alpha*beta)
    # travel_time = np.random.gamma(shape=5, scale=alpha*beta)
    # travel_time = max(beta * 5, travel_time)
    travel_time = sum(intervals[3:]) * np.random.uniform(0, 1)
    travel_time = max(alpha * beta, travel_time)

    return alpha, beta, h, c, total, intervals, travel_time

import pandas as pd

df = pd.DataFrame(columns=['h', 'c', 'travel_time', 'total', 'intervals', 'alpha', 'beta'])

for i in range(10000):
    alpha, beta, h, c, total, intervals, travel_time = get_realized_data(TEST_CONFIG)
    row = {'h': h, 'c': c, 'travel_time': travel_time, 'total': total, 'intervals': intervals, 'alpha': alpha, 'beta': beta}
    df.loc[i] = row



## Test

In [7]:
from stable_baselines3 import PPO
model = PPO.load(methods.file_path(PPO_MODEL_NAME, 'models'))

env = Env(config=TEST_CONFIG)
costs = {}
u_rl = {}
observed_n = {}
print(env._get_info())
for i in range(len(df)):
    row = df.iloc[i]
    state, _ = env.reset(row=row)
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(state)  
        state, reward, done, _, info = env.step(action)
        total_reward += reward
    costs[i] = -total_reward
    u_rl[i] = env.cur_time + env.travel_time
    observed_n[i] = env.final_observed_n
    # print(f"Episode {i} reward: {total_reward} | h = {info['state']['h']}, c = {info['state']['c']}")
    print(f"Episode {i} reward: {total_reward} | cur_time = {info['cur_time']}")

df['rl_cost'] = costs
df['u_rl'] = u_rl
df['rl_var_n_explore'] = observed_n
print(f"Average cost: {df['rl_cost'].mean()}")

{'config': {'alpha_range': [3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9], 'beta_range': [1.0, 1.5, 2.0, 2.5], 'h_range': [0.5], 'c_range': [25], 'total': range(10, 40), 'travel_time': 'uniform'}, 'alpha': np.float64(3.4), 'beta': np.float64(2.0), 'h': np.float64(0.5), 'c': np.int64(25), 'total': 24, 'travel_time': np.float64(107.13242924315836), 'intervals': [17.323180460778612, 1.7631822760106577, 4.2056968829869055, 8.83023566251302, 2.730035774527491, 5.165277011050095, 3.784856135025796, 8.482838274350136, 5.027259939984126, 11.105233038445675, 2.492328809445304, 6.8970812576159535, 5.525487597962424, 2.7831488922277603, 8.356872674260813, 7.909904382843607, 10.528614033296703, 8.892428609177868, 5.296097351503226, 10.171358406615578, 4.770600687097298, 5.314807531153056, 14.465395750547684, 2.4159682407894736], 'cum_sum_intervals': array([ 17.32318046,  19.08636274,  23.29205962,  32.12229528,
        34.85233106,  40.01760807,  43.8024642 ,  52.28530248,
        57.31256242,  68.

## Add Comparison Metrics

In [9]:

from pandarallel import pandarallel

# Initialize pandarallel with progress bar enabled
pandarallel.initialize(progress_bar=True)


for i in [3, 5]:
    print(f"Oracle cost at fix n = {i}")
    df[f'oracle_cost_fix_n_{i}'] = df.parallel_apply(lambda row: methods.cal_oracle_cost_fix_n(alpha=row['alpha'],\
                                                                                     beta=row['beta'],\
                                                                                     intervals=row['intervals'],\
                                                                                     h=row['h'],\
                                                                                     c=row['c'],\
                                                                                     travel_time=row['travel_time'],\
                                                                                     n=i), axis=1)
    
    print(f"PTO cost at fix n = {i}")
    df[f'PTO_cost_fix_n_{i}'] = df.parallel_apply(lambda row: methods.cal_PTO_cost_fix_n(intervals=row['intervals'],\
                                                                                     h=row['h'],\
                                                                                     c=row['c'],\
                                                                                     travel_time=row['travel_time'],\
                                                                                     n=i), axis=1)
                                                                                
print(f"Oracle cost at variable n")
cost_and_explre = df.parallel_apply(lambda row: methods.cal_oracle_cost_var_n(alpha=row['alpha'],\
                                                                                     beta=row['beta'],\
                                                                                     intervals=row['intervals'],\
                                                                                     h=row['h'],\
                                                                                     c=row['c'],\
                                                                                     travel_time=row['travel_time']),result_type='expand', axis=1)

df['oracle_cost_var_n'], df['oracle_var_n_explore'] = cost_and_explre[0], cost_and_explre[1]

print(f"PTO cost at variable n")
cost_and_explre = df.parallel_apply(lambda row: methods.cal_PTO_cost_var_n(intervals=row['intervals'],\
                                                                                     h=row['h'],\
                                                                                     c=row['c'],\
                                                                                     travel_time=row['travel_time']),result_type='expand', axis=1)

df['PTO_cost_var_n'], df['PTO_var_n_explore'] = cost_and_explre[0], cost_and_explre[1]

df.head()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Oracle cost at fix n = 3


PTO cost at fix n = 3


Oracle cost at fix n = 5


PTO cost at fix n = 5


Oracle cost at variable n


PTO cost at variable n


,h,c,travel_time,total,intervals,alpha,beta,rl_cost,u_rl,rl_var_n_explore,oracle_cost_fix_n_3,PTO_cost_fix_n_3,oracle_cost_fix_n_5,PTO_cost_fix_n_5,oracle_cost_var_n,oracle_var_n_explore,PTO_cost_var_n,PTO_var_n_explore
0,0.5,25,112.052318,33,"[2.564511919044639, 9.514325855495155, 6.24951...",3.6,1.5,19.312923,133.435394,3,3.568807,25.000000,4.094239,1.182080,10.104409,10.0,13.311923,7.0
1,0.5,25,44.443818,21,"[3.3053173426932076, 0.8321828229755426, 2.834...",3.6,1.0,7.369855,52.345439,3,3.437331,7.834656,4.862882,5.860796,4.862882,5.0,7.834656,3.0
2,0.5,25,10.604424,12,"[9.009832488275858, 1.9544800422690054, 5.0568...",3.1,2.5,24.734388,27.693650,3,3.311238,13.440571,5.060034,11.272773,6.224313,9.0,9.229615,8.0
3,0.5,25,21.519832,12,"[2.422082888418795, 4.898339929966356, 3.41164...",3.4,1.0,25.000000,33.325109,3,25.000000,25.000000,25.000000,25.000000,0.164153,2.0,25.000000,3.0
4,0.5,25,148.916093,27,"[3.30599311631303, 2.8876274339954944, 12.4459...",3.3,2.0,0.589328,168.176997,3,0.899988,0.899988,25.000000,25.000000,7.018323,2.0,0.899988,3.0


## Visualize Results

In [10]:
import plotly.graph_objects as go

all_costs = [x for x in df.columns if 'cost' in x] + ['oracle_var_n_explore', 'rl_var_n_explore', 'PTO_var_n_explore']

# Calculate means
means = df[all_costs + ['h', 'c']].mean()

# Plot
fig = go.Figure(data=[go.Bar(x=means.index, y=means.values)])
fig.update_layout(title="Mean Values of Costs, H, and C", yaxis_title="Mean Value")
fig.show()

# Calculate medians
medians = df[all_costs + ['h', 'c']].median()

# Plot
fig = go.Figure(data=[go.Bar(x=medians.index, y=medians.values)])
fig.update_layout(title="Median Values of Costs, H, and C", yaxis_title="Median Value")
fig.show()

# Create a dataframe with two rows: one for means and one for medians
summary_df = pd.DataFrame({'Mean': means, 'Median': medians}).T
summary_df.head()


,rl_cost,oracle_cost_fix_n_3,PTO_cost_fix_n_3,oracle_cost_fix_n_5,PTO_cost_fix_n_5,oracle_cost_var_n,PTO_cost_var_n,oracle_var_n_explore,rl_var_n_explore,PTO_var_n_explore,h,c
Mean,32.682631,10.791403,18.172271,12.097820,16.961909,9.404910,11.724587,11.0186,3.1188,10.9835,0.5,25.0
Median,25.000000,8.339546,20.268193,9.241533,18.965261,7.447588,8.184055,9.0000,3.0000,9.0000,0.5,25.0


In [11]:
df.head(50)
tp = len(df[df['oracle_cost_fix_n_5'] == 25])
print(tp)

2382
